In [4]:
from bs4 import BeautifulSoup as BS
import requests
from pandas import ExcelWriter
import re
import time
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
global fail_url_model_2,fail_url_komplektacii
import random
# Глобокий парсинг параметров авто с www.drom.ru

def connect_url(url):

    proxies = [{"http" : 'https://194.67.217.115:9478'},
               {"http" : 'https://194.67.213.80:9146'},
               {"http" : 'https://194.67.213.179:9950'},
               {"http" : 'https://193.233.62.90:9457'},
               {"http" : 'https://194.67.214.187:9409'},
               {"http" : 'https://194.67.213.226:9348'}]

    HEADERS = {
            'Host': 'www.drom.ru',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:69.0) Gecko/20100101 Firefox/69.0',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
            'Accept-Language': 'ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3',
            'Accept-Encoding': 'utf-8',
            'Connection': 'keep-alive',
            'Cookie': 'ring=1bcccadQY2rDDGCzTFzfFwBM0nL0A0a1; cookie_cityid=40; cookie_regionid=26; my_geo=26; PHPSESSID=42e4ee865e6b15eb6dd62b5af37e6ec2; dr_df=1; segSession=ImQwYmI1MmJjN2IxNDdmMjhmYTJmZGJjYzY2OWYxZmU3bm90QXV0aDFiY2NjYWRRWTJyRERHQ3pURnpmRndCTTBuTDBBMGExIl81MjI3ZTlhYjYyOTJmNzY0ZDhhOGRhODUwNjExMDUzOQ; _ga_1G91VLKB2K=GS1.1.1684355234.1.0.1684355234.60.0.0; _ga=GA1.2.1445469351.1684355234; _gid=GA1.2.1053840647.1684355235; _gat=1',

            'Upgrade-Insecure-Requests': '1',
            'Cache-Control': 'max-age=0'
        }

    proxi=random.choice(proxies)

    response = requests.get(url,  headers=HEADERS, proxies=proxi).text

    return response

def parse_katalog_2(url):
    link_2 = url
    response = connect_url(link_2)
    soup = BS(response, "lxml")
    bs4_katalog = soup.find_all('a', attrs={'class': 'e1ei9t6a1 css-1x6lzas ezhoka60'})
    href_katalog = []
    for i in bs4_katalog:
        href_katalog.append(i.attrs.get('href'))
    return href_katalog

def parse_model_row(url): #для каталога так же
    link_2 = url
    response = connect_url(link_2)
    soup = BS(response, "lxml")
    bs4_model_row = soup.find('div', attrs={'class': 'css-18clw5c ehmqafe0'}).find_all('a')
    href_model_row = []
    for i in bs4_model_row:
        href_model_row.append(i.attrs.get('href'))
    return href_model_row

def parse_model_in_model_row(url): #для каталога так же
    try:
        link_2 = url
        response = connect_url(link_2)
        soup = BS(response, "lxml")
        country_soup = soup.find_all('div', attrs={'class': 'css-pyemnz e1ei9t6a4'})

        href_models_in_model_row = []
        for step in country_soup:
            country = step.find('div').get('id')
            bs4_model_in_model_row = step.find_all('a', attrs={'class': 'e1ei9t6a1 css-1x6lzas ezhoka60'})

            for i in bs4_model_in_model_row:
                d = []
                d.append(country)
                d.append(url + i.attrs.get('href'))
                href_models_in_model_row.append(d)
                # href_models_in_model_row.append(url + i.attrs.get('href'))
            # print(href_models_in_model_row)
        return href_models_in_model_row
    except:
        try:
            link_2 = url
            response = connect_url(link_2)
            soup = BS(response, "lxml")
            country_soup = soup.find_all('div', attrs={'class': 'css-pyemnz e1ei9t6a4'})

            href_models_in_model_row = []
            for step in country_soup:
                country = step.find('div').get('id')
                bs4_model_in_model_row = step.find_all('a', attrs={'class': 'e1ei9t6a1 css-1x6lzas ezhoka60'})

                for i in bs4_model_in_model_row:
                    d = []
                    d.append(country)
                    d.append(url + i.attrs.get('href'))
                    href_models_in_model_row.append(d)
                    # href_models_in_model_row.append(url + i.attrs.get('href'))
                # print(href_models_in_model_row)
            return href_models_in_model_row
        except:
            fail_url_model_2.append(url)
            #print(url, '   FAILED')

def parse_komplektacii(url):
    try:
        link_2 = url[1]
        response = connect_url(link_2)
        soup = BS(response, "lxml")
        bs4_komplektacii = soup.find_all('tr', attrs={'class': 'b-table__row b-table_align_center b-table__row_border_bottom b-table__row_padding_size-s'})
        #print(bs4_komplektacii[0].find_all('a')[0].attrs.get('href'))
    except:
        try:
            link_2 = url[1]
            response = connect_url(link_2)
            soup = BS(response, "lxml")
            bs4_komplektacii = soup.find_all('tr', attrs={
                'class': 'b-table__row b-table_align_center b-table__row_border_bottom b-table__row_padding_size-s'})
        except:
            bs4_komplektacii = ''
            fail_url_komplektacii.append(url)
    href_bs4_komplektacii = []
    for i in bs4_komplektacii:
        href = i.find_all('a')[0].attrs.get('href')
        if len(href)>5 and 'cukb-czea' not in href:
            href_bs4_komplektacii.append('https://www.drom.ru'+href)
        else:
            pass
    hrefs_2 = list(set(href_bs4_komplektacii))

    country = url[0]
    country_url = []
    for j in hrefs_2:
        d = []
        d.append(country)
        d.append(j)
        country_url.append(d)
    return country_url




def parse_harakteristics(url):

    parametr = ['Марка','Модель','Модельный ряд для','url']
    #znachenie = [url[1].split('/')[5], url[1].split('/')[6], url[0], url[1]]

    if 'lcv' in url[1]:
        znachenie = [url[1].split('/')[5], url[1].split('/')[6], url[0], url[1]]
    else:
        znachenie = [url[1].split('/')[4], url[1].split('/')[5], url[0], url[1]]

    link_2 = url[1]
    response = connect_url(link_2)
    soup = BS(response, "lxml")
    bs4_komplektacii = soup.find('tbody').find_all('tr')


    for n in bs4_komplektacii:
        try:
            symbol = n.find('use').attrs.get('xlink:href')
            if len(n.get_text()) < 100:
                if symbol == '#yes':
                    symbol = 'да'

                    parametr.append(n.get_text())
                    znachenie.append(symbol)

                elif symbol == '#option':
                    symbol = 'опция производителя'

                    parametr.append(n.get_text())
                    znachenie.append(symbol)
            else:
                # list_values = n.get_text().split('      ')
                # sorted_list = sorted(list_values, key=len)
                # parametr_ = sorted_list[-2].strip()
                # znachenie_ = sorted_list[-1].strip()

                str_values = re.sub('          +', '9999999', n.get_text())
                list_values = str_values.split('9999999')
                list_values_2 = [x for x in list_values if x != '\n']
                parametr_ = list_values_2[0].strip()
                znachenie_ = list_values_2[1].strip()

                parametr.append(parametr_)
                znachenie.append(znachenie_)
        except:
            try:
                parametr_ = n.find_all('td')[0].get_text()
                if parametr_ != 'Период выпуска':
                    correctedstring = " ".join(n.find_all('td')[1].get_text().split())
                    if correctedstring == '—':
                        correctedstring = 'нет'
                        parametr.append(parametr_)
                        znachenie.append(correctedstring.split('Отзывы')[0])
                    elif correctedstring != '':
                        parametr.append(parametr_)
                        znachenie.append(correctedstring.split('Отзывы')[0])
                else:
                    h1_data = soup.find('h1').get_text().split(')')[-2]
                    h1_data = h1_data.split('(')[1]
                    parametr.append(parametr_)
                    znachenie.append(h1_data)
            except:
                pass
    df = pd.DataFrame({'parametr': parametr, 'znachenie': znachenie})
    return df

if __name__ == '__main__':
    #ВСЕ ссылки с ошибками
    fail_url_model_0_all = []
    fail_url_model_1_all = []
    fail_url_model_2_all = []
    fail_url_komplektacii_all = []


    href_katalog = parse_model_row('https://www.drom.ru/catalog/lcv/')
    print(href_katalog)
    print(len(href_katalog))


#ВОТ ТУТ МЕНЯЕШЬ ГРАНИЦУ (:1) ЭТО МАРКИ
    # 80:95
    #href_katalog_in_progress = href_katalog[96:]
    #проблемные
    href_katalog_in_progress = ['https://www.drom.ru/catalog/lcv/baic/','https://www.drom.ru/catalog/lcv/skywell/']

    schetchik = 0
    for marka in href_katalog_in_progress:

        #ссылки с ошибками
        fail_url_model_1 = []
        fail_url_model_2 = []
        fail_url_komplektacii = []
        ########

        start_time = time.time()
        schetchik += 1

        if 'lcv' in marka:
            file_marka = marka.split('/')[5]
        else:
            file_marka = marka.split('/')[4]

        print('\n', file_marka ,' <<',schetchik ,' of ',len(href_katalog_in_progress), '>>')


        href_katalog = marka.split()
        print(href_katalog)
        href_model_row_all = []
        for i in href_katalog:
            try:
                href_model_row = parse_model_row(i)
                href_model_row_all = href_model_row_all + href_model_row
                print(i+"  ok")
            except:
                fail_url_model_0_all.append(i)
                print(i + "  NET")
                pass
        print(href_model_row_all)
        print(len(href_model_row_all))


        #href_model_row_all = href_model_row_all[:1]
        href_models_in_model_row_all = []
        for i in href_model_row_all:
            try:
                href_models_in_model_row = parse_model_in_model_row(i)
                href_models_in_model_row_all = href_models_in_model_row_all + href_models_in_model_row
                #print(i + "  ok")
            except:
                fail_url_model_1_all.append(i)
                print(i + "  NET")

        #print(href_models_in_model_row_all)
        print(len(href_models_in_model_row_all))


        #href_models_in_model_row_all = href_models_in_model_row_all[:]
        href_komplektacii_all = []
        for i in href_models_in_model_row_all:
            try:
                href_komplektacii = parse_komplektacii(i)
                href_komplektacii_all = href_komplektacii_all + href_komplektacii
                #print(i + "  ok")
            except:
                fail_url_model_2_all.append(i)
                print(i + "  NET")

        #print(href_komplektacii_all)
        print(len(href_komplektacii_all))


        df_all = pd.DataFrame()
        href_komplektacii_all = href_komplektacii_all[:]
        k = 1
        kolvo = len(href_komplektacii_all)
        for i in href_komplektacii_all:
            if k % 100 == 0:
                print(k,'  из  ', kolvo)
            k += 1

            try:
                df = parse_harakteristics(i)
                if df_all.empty:
                    df_all = df
                else:
                    df_all = df_all.merge(df, how="outer", on='parametr', suffixes=('_x', '_y'))
            except:
                fail_url_komplektacii.append(i)



        #print(fail_url)
        if df_all.size != 0:
            print('{0} готова {1}'.format(file_marka,df_all.shape))
            df_all = df_all.transpose()
            filename = r'D:\Pycharm_projects\RGS\dron\new_parsing_with_miss\marks_G\{}.xlsx'.format(file_marka)
            df_all.to_excel(filename, index=False, header=False)
            print("--- %s seconds ---" % (time.time() - start_time))


        print('Необработанные ссылки --', len(fail_url_model_1),' шт')
        print('Необработанные ссылки --', len(fail_url_model_2),' шт')
        print('Необработанные ссылки --', len(fail_url_komplektacii),' шт')
        fail_url_model_1_all = fail_url_model_1_all + fail_url_model_1
        fail_url_model_2_all = fail_url_model_2_all + fail_url_model_2
        fail_url_komplektacii_all = fail_url_komplektacii_all + fail_url_komplektacii

['https://www.drom.ru/catalog/lcv/citroen/', 'https://www.drom.ru/catalog/lcv/daihatsu/', 'https://www.drom.ru/catalog/lcv/fiat/', 'https://www.drom.ru/catalog/lcv/ford/', 'https://www.drom.ru/catalog/lcv/honda/', 'https://www.drom.ru/catalog/lcv/hyundai/', 'https://www.drom.ru/catalog/lcv/isuzu/', 'https://www.drom.ru/catalog/lcv/kia/', 'https://www.drom.ru/catalog/lcv/mazda/', 'https://www.drom.ru/catalog/lcv/mercedes-benz/', 'https://www.drom.ru/catalog/lcv/mitsubishi/', 'https://www.drom.ru/catalog/lcv/nissan/', 'https://www.drom.ru/catalog/lcv/peugeot/', 'https://www.drom.ru/catalog/lcv/renault/', 'https://www.drom.ru/catalog/lcv/toyota/', 'https://www.drom.ru/catalog/lcv/volkswagen/', 'https://www.drom.ru/catalog/lcv/gaz/', 'https://www.drom.ru/catalog/lcv/lada/', 'https://www.drom.ru/catalog/lcv/uaz/', 'https://www.drom.ru/catalog/lcv/avia/', 'https://www.drom.ru/catalog/lcv/baic/', 'https://www.drom.ru/catalog/lcv/bajaj/', 'https://www.drom.ru/catalog/lcv/barkas/', 'https://www

In [2]:
fail_url_model_0_all

[]

In [10]:
with open(r'D:\Pycharm_projects\RGS\dron\new_parsing_with_miss\miss\miss_models_1.txt', 'a') as f:
    for line in fail_url_model_1_all:
        f.write("%s\n" % line)
with open(r'D:\Pycharm_projects\RGS\dron\new_parsing_with_miss\miss\miss_models_2.txt', 'a') as f:
    for line in fail_url_model_2_all:
        f.write("%s\n" % line)
with open(r'D:\Pycharm_projects\RGS\dron\new_parsing_with_miss\miss\miss_komplektacii.txt', 'a') as f:
    for line in fail_url_komplektacii_all:
        f.write("%s\n" % line)

In [7]:
a = ['https://www.drom.ru/catalog/audi/', 'https://www.drom.ru/catalog/bmw/', 'https://www.drom.ru/catalog/chevrolet/', 'https://www.drom.ru/catalog/ford/', 'https://www.drom.ru/catalog/honda/', 'https://www.drom.ru/catalog/hyundai/', 'https://www.drom.ru/catalog/kia/', 'https://www.drom.ru/catalog/lexus/', 'https://www.drom.ru/catalog/mazda/', 'https://www.drom.ru/catalog/mitsubishi/', 'https://www.drom.ru/catalog/nissan/', 'https://www.drom.ru/catalog/opel/', 'https://www.drom.ru/catalog/renault/', 'https://www.drom.ru/catalog/skoda/', 'https://www.drom.ru/catalog/subaru/', 'https://www.drom.ru/catalog/suzuki/', 'https://www.drom.ru/catalog/toyota/', 'https://www.drom.ru/catalog/volkswagen/', 'https://www.drom.ru/catalog/lada/', 'https://www.drom.ru/catalog/ac/', 'https://www.drom.ru/catalog/acura/', 'https://www.drom.ru/catalog/aito/', 'https://www.drom.ru/catalog/alfa_romeo/', 'https://www.drom.ru/catalog/alpina/', 'https://www.drom.ru/catalog/aro/', 'https://www.drom.ru/catalog/aston_martin/', 'https://www.drom.ru/catalog/avatr/', 'https://www.drom.ru/catalog/baic/', 'https://www.drom.ru/catalog/baojun/', 'https://www.drom.ru/catalog/barkas/', 'https://www.drom.ru/catalog/baw/', 'https://www.drom.ru/catalog/bedford/', 'https://www.drom.ru/catalog/bentley/', 'https://www.drom.ru/catalog/brilliance/', 'https://www.drom.ru/catalog/bugatti/', 'https://www.drom.ru/catalog/buick/', 'https://www.drom.ru/catalog/byd/', 'https://www.drom.ru/catalog/cadillac/', 'https://www.drom.ru/catalog/changan/', 'https://www.drom.ru/catalog/changhe/', 'https://www.drom.ru/catalog/chery/', 'https://www.drom.ru/catalog/chrysler/', 'https://www.drom.ru/catalog/citroen/', 'https://www.drom.ru/catalog/cupra/', 'https://www.drom.ru/catalog/dacia/', 'https://www.drom.ru/catalog/dadi/', 'https://www.drom.ru/catalog/daewoo/', 'https://www.drom.ru/catalog/daihatsu/', 'https://www.drom.ru/catalog/datsun/', 'https://www.drom.ru/catalog/dayun/', 'https://www.drom.ru/catalog/delorean/', 'https://www.drom.ru/catalog/denza/', 'https://www.drom.ru/catalog/derways/', 'https://www.drom.ru/catalog/dodge/', 'https://www.drom.ru/catalog/dongfeng/', 'https://www.drom.ru/catalog/dw_hower/', 'https://www.drom.ru/catalog/evolute/', 'https://www.drom.ru/catalog/cheryexeed/', 'https://www.drom.ru/catalog/faw/', 'https://www.drom.ru/catalog/ferrari/', 'https://www.drom.ru/catalog/fiat/', 'https://www.drom.ru/catalog/fisker/', 'https://www.drom.ru/catalog/forthing/', 'https://www.drom.ru/catalog/foton/', 'https://www.drom.ru/catalog/gac/', 'https://www.drom.ru/catalog/geely/', 'https://www.drom.ru/catalog/genesis/', 'https://www.drom.ru/catalog/geo/', 'https://www.drom.ru/catalog/gmc/', 'https://www.drom.ru/catalog/great_wall/', 'https://www.drom.ru/catalog/hafei/', 'https://www.drom.ru/catalog/haima/', 'https://www.drom.ru/catalog/haval/', 'https://www.drom.ru/catalog/hawtai/', 'https://www.drom.ru/catalog/hiphi/', 'https://www.drom.ru/catalog/hongqi/', 'https://www.drom.ru/catalog/hozon/', 'https://www.drom.ru/catalog/hummer/', 'https://www.drom.ru/catalog/infiniti/', 'https://www.drom.ru/catalog/iran_khodro/', 'https://www.drom.ru/catalog/isuzu/', 'https://www.drom.ru/catalog/iveco/', 'https://www.drom.ru/catalog/jac/', 'https://www.drom.ru/catalog/jaguar/', 'https://www.drom.ru/catalog/jeep/', 'https://www.drom.ru/catalog/jetour/', 'https://www.drom.ru/catalog/jetta/', 'https://www.drom.ru/catalog/kaiyi/', 'https://www.drom.ru/catalog/lamborghini/', 'https://www.drom.ru/catalog/lancia/', 'https://www.drom.ru/catalog/land_rover/', 'https://www.drom.ru/catalog/landwind/', 'https://www.drom.ru/catalog/leapmotor/', 'https://www.drom.ru/catalog/li/', 'https://www.drom.ru/catalog/lifan/', 'https://www.drom.ru/catalog/lincoln/', 'https://www.drom.ru/catalog/lotus/', 'https://www.drom.ru/catalog/lucid/', 'https://www.drom.ru/catalog/luxgen/', 'https://www.drom.ru/catalog/lynk_and_co/', 'https://www.drom.ru/catalog/marussia/', 'https://www.drom.ru/catalog/maserati/', 'https://www.drom.ru/catalog/maxus/', 'https://www.drom.ru/catalog/maybach/', 'https://www.drom.ru/catalog/mclaren/', 'https://www.drom.ru/catalog/mercedes-benz/', 'https://www.drom.ru/catalog/mercury/', 'https://www.drom.ru/catalog/mg/', 'https://www.drom.ru/catalog/mini/', 'https://www.drom.ru/catalog/mitsuoka/', 'https://www.drom.ru/catalog/nio/', 'https://www.drom.ru/catalog/oldsmobile/', 'https://www.drom.ru/catalog/omoda/', 'https://www.drom.ru/catalog/oshan/', 'https://www.drom.ru/catalog/pagani/', 'https://www.drom.ru/catalog/peugeot/', 'https://www.drom.ru/catalog/piaggio/', 'https://www.drom.ru/catalog/plymouth/', 'https://www.drom.ru/catalog/polestar/', 'https://www.drom.ru/catalog/pontiac/', 'https://www.drom.ru/catalog/porsche/', 'https://www.drom.ru/catalog/radar/', 'https://www.drom.ru/catalog/ram/', 'https://www.drom.ru/catalog/ravon/', 'https://www.drom.ru/catalog/renault_samsung/', 'https://www.drom.ru/catalog/rimac/', 'https://www.drom.ru/catalog/rivian/', 'https://www.drom.ru/catalog/roewe/', 'https://www.drom.ru/catalog/rolls-royce/', 'https://www.drom.ru/catalog/rover/', 'https://www.drom.ru/catalog/saab/', 'https://www.drom.ru/catalog/saipa/', 'https://www.drom.ru/catalog/saturn/', 'https://www.drom.ru/catalog/scion/', 'https://www.drom.ru/catalog/seat/', 'https://www.drom.ru/catalog/seres/', 'https://www.drom.ru/catalog/shuanghuan/', 'https://www.drom.ru/catalog/skywell/', 'https://www.drom.ru/catalog/smart/', 'https://www.drom.ru/catalog/soueast/', 'https://www.drom.ru/catalog/ssang_yong/', 'https://www.drom.ru/catalog/swm/', 'https://www.drom.ru/catalog/tank/', 'https://www.drom.ru/catalog/tata/', 'https://www.drom.ru/catalog/tesla/', 'https://www.drom.ru/catalog/tianma/', 'https://www.drom.ru/catalog/tianye/', 'https://www.drom.ru/catalog/volvo/', 'https://www.drom.ru/catalog/vortex/', 'https://www.drom.ru/catalog/voyah/', 'https://www.drom.ru/catalog/weltmeister/', 'https://www.drom.ru/catalog/wuling/', 'https://www.drom.ru/catalog/xpeng/', 'https://www.drom.ru/catalog/zeekr/', 'https://www.drom.ru/catalog/zotye/', 'https://www.drom.ru/catalog/zx/', 'https://www.drom.ru/catalog/aurus/', 'https://www.drom.ru/catalog/bogdan/', 'https://www.drom.ru/catalog/gaz/', 'https://www.drom.ru/catalog/doninvest/', 'https://www.drom.ru/catalog/zaz/', 'https://www.drom.ru/catalog/zil/', 'https://www.drom.ru/catalog/zis/', 'https://www.drom.ru/catalog/izh/', 'https://www.drom.ru/catalog/luaz/', 'https://www.drom.ru/catalog/moskvitch/', 'https://www.drom.ru/catalog/tagaz/', 'https://www.drom.ru/catalog/uaz/']

In [8]:
a.index('https://www.drom.ru/catalog/mitsubishi/')

9